<a href="https://colab.research.google.com/github/Rameshkumar789/Computational-Menthods-INFO-5731/blob/main/Citation-based%20Summarization/%20Similarity_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving project_50.csv to project_50.csv


In [ ]:
df=pd.read_csv('/content/project_50.csv')

In [ ]:
df

,Unnamed: 0,SourceID,Title,TextWhereRefMention
0,0,176908,Urban wages and labor market agglomeration,"Wheaton and Lewis, Ref{target=#b33, content=},..."
1,1,176908,NaN,"Glaeser and MarÃ©, Ref{target=#b9, content=}, ..."
2,2,176908,NaN,"[Wheeler, Ref{target=#b34, content=}, provides..."
3,3,176908,The Wage Effects of Residential Location and C...,"First, empirical evidence suggests that women ..."
4,4,176908,Race and Gender in the Labor Market,"Second, women typically work fewer hours and y..."
...,...,...,...,...
102024,102024,30685534,Early Learning Standards: Results from a Natio...,Most US states have now developed statements o...
102025,102025,30685534,Inside the Content of InfantToddler Early Lear...,Most US states have now developed statements o...
102026,102026,30685534,Early Learning Guidelines for Infants and Todd...,[Peak bodies in the US have published recommen...
102027,102027,30685534,Early Learning Guidelines for Infants and Todd...,the process used to develop SÃ­olta: The Natio...


## Data Cleaning

In [ ]:
import re

In [ ]:
cleaned_text=[]

for i in range(len(df)):
  temp = re.sub('Ref{.*?}', '', df['TextWhereRefMention'][i])
  temp = re.sub(r'[^\w\s]', '', temp)
  temp= re.sub(' +', ' ', temp).rstrip()
  cleaned_text.append(temp)

In [ ]:
df['cleaned_text']=cleaned_text

In [ ]:
df

,Unnamed: 0,SourceID,Title,TextWhereRefMention,cleaned_text
0,0,176908,Urban wages and labor market agglomeration,"Wheaton and Lewis, Ref{target=#b33, content=},...",Wheaton and Lewis find evidence of increasing ...
1,1,176908,NaN,"Glaeser and MarÃ©, Ref{target=#b9, content=}, ...",Glaeser and MarÃ find a robust urban wage prem...
2,2,176908,NaN,"[Wheeler, Ref{target=#b34, content=}, provides...",Wheeler provides empirical support for a model...
3,3,176908,The Wage Effects of Residential Location and C...,"First, empirical evidence suggests that women ...",First empirical evidence suggests that women a...
4,4,176908,Race and Gender in the Labor Market,"Second, women typically work fewer hours and y...",Second women typically work fewer hours and ye...
...,...,...,...,...,...
102024,102024,30685534,Early Learning Standards: Results from a Natio...,Most US states have now developed statements o...,Most US states have now developed statements o...
102025,102025,30685534,Inside the Content of InfantToddler Early Lear...,Most US states have now developed statements o...,Most US states have now developed statements o...
102026,102026,30685534,Early Learning Guidelines for Infants and Todd...,[Peak bodies in the US have published recommen...,Peak bodies in the US have published recommend...
102027,102027,30685534,Early Learning Guidelines for Infants and Todd...,the process used to develop SÃ­olta: The Natio...,the process used to develop SÃolta The Nationa...


In [ ]:
df.to_csv('Cleaned_data.csv')

## Grouping the Data

In [ ]:
x = df.groupby(['SourceID']).size().reset_index(name='count')
lis_id = x['SourceID'].values.tolist()

In [ ]:
len(lis_id)

903

## Model

## Cosine-similarity using Scibert Embedding 

In [ ]:
!pip install transformers

In [ ]:
import torch

from transformers import BertModel, BertTokenizer
import numpy as np
from scipy.spatial.distance import cosine

In [ ]:
scibert_model = BertModel.from_pretrained("allenai/scibert_scivocab_uncased",
                                  output_hidden_states=True)
scibert_tokenizer = BertTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

In [ ]:
def get_embedding(text):
    
    b_model=scibert_model
    b_tokenizer=scibert_tokenizer

    encoded_dict = b_tokenizer.encode_plus(
                        text,                      
                        add_special_tokens = True, 
                        return_tensors = 'pt',     
                )

    input_ids = encoded_dict['input_ids']
    
    b_model.eval()

    bert_outputs = b_model(input_ids)
    
    with torch.no_grad():

        outputs = b_model(input_ids)

        hidden_states = outputs[2]

    # Hidden_states: shape [13 x 1 x <sentence length> x 768]
    # 'Token_vecs' is a tensor with shape [<sent length> x 768]
    token_vecs = hidden_states[-2][0]

    sentence_embedding = torch.mean(token_vecs, dim=0)

    sentence_embedding = sentence_embedding.detach().numpy()
    
    return sentence_embedding

In [ ]:
similarity = []

for i in range(8, len(lis_id)):
  
  temp = df.loc[df['SourceID'] == lis_id[i]]
  temp=temp.drop_duplicates(subset='cleaned_text', keep="first")
  temp = temp.reset_index(drop=True)
  record_similarity = []
  for l in range(len(temp)):
    emb_query = get_embedding(temp['cleaned_text'][l])
    row_simil=[]
    for k in range(0, l+1):
      if (k==l):
        row_simil.append(1)
      else:
        row_simil.append(0)
    for j in range(l+1, len(temp)):
      emb_A = get_embedding(temp['cleaned_text'][j])
      sim_query_A = 1 - cosine(emb_query, emb_A)
      row_simil.append(sim_query_A)
  
    record_similarity.append(row_simil)
  print(i, "Complete")
  similarity.append(record_similarity)

8 Complete


In [ ]:
import csv
with open("Similarity.csv", 'w') as f:
   writer = csv.writer(f, delimiter=',')
   writer.writerows(similarity)